<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/ex11notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex11notebookC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


----
## $K$-平均法の実験
----

K-平均法によるクラスタリングの実験をやってみましょう．



まずはいつものように準備から．



In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

K-meansクラスタリングのための関数を定義しておきます．

In [ ]:
## セントロイドの初期化
#
def initCentroid(X, centroid, seed=None):
    assert X.shape[1] == centroid.shape[1]
    K = centroid.shape[0]
    # 学習データからランダムに K 個を選択して初期セントロイドとする
    N = X.shape[0]
    idx = np.arange(N, dtype=int)
    if seed is not None:
        np.random.seed(seed)
    np.random.shuffle(idx)
    centroid[:] = X[idx[:K], :]

## データをクラスタに割り振る
#
def assignCluster(X, centroid, label):
    assert X.shape[1] == centroid.shape[1] and X.shape[0] == label.shape[0]
    K = centroid.shape[0]
    N = X.shape[0]
    sqe = 0.0
    for n in range(N):
        # 各セントロイドとの距離の二乗を計算
        d = np.sum((X[n, :] - centroid)**2, axis=1)
        # 距離最小のクラスタへ割り振る
        i = np.argmin(d)
        label[n] = i
        sqe += d[i]

    return sqe/N  # 割り振られたセントロイドとの距離の二乗の平均

## セントロイドを計算し直す
#
def updateCentroid(X, centroid, label):
    assert X.shape[1] == centroid.shape[1] and X.shape[0] == label.shape[0]
    K = centroid.shape[0]
    for ik in range(K):
        # ik 番目のクラスタに割り当てられたデータの平均をそのクラスタの新しいセントロイドとする
        centroid[ik, :] = np.mean(X[label==ik, :], axis=0)

画像のクラスタリングの実験で使う，可視化用の関数を定義しておきます．

In [ ]:
#####  データの最初の nx x ny 枚を可視化
#
def mosaicImage(dat, nx, ny, nrow=64, ncol=64, gap=4):

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            if iy*nx+ix >= dat.shape[0]:
                break
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = dat[iy*nx+ix, :].reshape((nrow, ncol))
            
    return img


#####  セントロイドとともにデータの最初の n 枚を可視化
#
def mosaicImage2(X, centroid, index, n, nrow=64, ncol=64, gap=4):

    nx = n + 2
    ny = centroid.shape[0]
    
    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        ix = 0
        ltx = ix*(ncol + gap) + gap
        img[lty:lty+nrow, ltx:ltx+ncol] = centroid[iy, :].reshape((nrow, ncol))
        dat = X[index == iy, :]
        for ix in range(2, nx):
            jx = ix - 2
            if jx >= dat.shape[0]:
                break
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = dat[jx, :].reshape((nrow, ncol))
            
    return img


---
### ［実験1］ 3科目の試験の得点

「数学」「物理」「情報」の3科目の試験の点数を100人分集めたデータをクラスタリングしてみましょう．

In [ ]:
# 数物情データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/PIP/mpi100-mac.csv
dfMPI = pd.read_csv('mpi100-mac.csv', index_col=0)
datMPI = dfMPI.to_numpy().astype(float)
dfMPI

In [ ]:
## 数物情データの K-means クラスタリング

K = 3  # クラスタ数
nitr = 10  # K-means 法の繰り返し回数

X = datMPI
N, D = X.shape
centroid = np.empty((K, D)) # セントロイド
label = np.empty(N, dtype=int) # 各学習データの所属するセントロイドの番号

# セントロイドを初期化
initCentroid(X, centroid)  

print('#itr  msqe    Nk')
for i in range(nitr):
    # データを各クラスタに割り振る
    msqe = assignCluster(X, centroid, label)
    # 各クラスタに割り振られたデータの数と，誤差の値を出力
    Nk = np.empty(K, dtype=int)
    for ik in range(K):
        Nk[ik] = np.sum(label == ik)
    print(f'{i}    {msqe:.2f}   {Nk}')    
    # セントロイドを更新
    updateCentroid(X, centroid, label)

# セントロイドの値を出力
for ik in range(K):
    print(f'クラスタ {ik} のセントロイド:  ({centroid[ik, 0]:.1f}, {centroid[ik, 1]:.1f}, {centroid[ik, 2]:.1f})')

#### ★★★ やってみよう ★★★

- クラスタリングを実行して得られる結果を観察しよう．実行のたびに初期値が変わり，学習の結果も変わるので，何度か実行し直してみよう．
- 3つのクラスタに分けられたそれぞれの集団の得点の傾向を，セントロイドの値から考察してみよう．「他の集団に比べて○○の点が高いが△△の点は低い」，etc.

---
### ［実験2］ 猫顔画像のクラスタリング

猫の顔画像をクラスタリングしてみましょう．
この実験で使っている猫画像は，他の目的に使ってはいけません．

In [ ]:
# 猫画像データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/PIP/cat131.npz
cat = np.load('cat131.npz')['cat131']
print(cat.shape) # 131枚，ひとつの画像は 64 x 64 = 4096 画素のグレイスケール画像

In [ ]:
print('### 猫画像 ###')
print(f'データ数 x 次元数 = {cat.shape[0]} x {cat.shape[1]}')

# 最初の 16 枚を表示 
img = mosaicImage(cat, 4, 4)
plt.axis('off')
plt.imshow(img, cmap = 'gray')
plt.show()   

学習を実行し，結果を可視化させてみましょう．

In [ ]:
## 猫画像データの K-means クラスタリング

K = 3  # クラスタ数
nitr = 10  # K-means 法の繰り返し回数

X = cat
N, D = X.shape
centroid = np.empty((K, D)) # セントロイド
label = np.empty(N, dtype=int) # 各学習データの所属するセントロイドの番号

# セントロイドを初期化
initCentroid(X, centroid)  

print('#itr  msqe    Nk')
for i in range(nitr):
    # データを各クラスタに割り振る
    msqe = assignCluster(X, centroid, label)
    # 各クラスタに割り振られたデータの数と，誤差の値を出力
    Nk = np.empty(K, dtype=int)
    for ik in range(K):
        Nk[ik] = np.sum(label == ik)
    print(f'{i}    {msqe:e}   {Nk}')    
    # セントロイドを更新
    updateCentroid(X, centroid, label)

# クラスタ毎のセントロイドと所属データを可視化
#     各行が一つのクラスタに対応，左端がセントロイド，右の画像はそのクラスタに所属する画像の一部
img = mosaicImage2(cat, centroid, label, 8)
plt.figure(figsize=(10,10))
plt.axis('off')
plt.imshow(img, cmap = 'gray')
plt.show()
s = '''
クラスタ毎のセントロイドと所属データを可視化
　各行が一つのクラスタに対応，左端がセントロイド，右の画像はそのクラスタに所属する画像の一部
'''
print(s)

#### ★★★ やってみよう★★★

上記では，猫画像を K = 3 の K-means法でクラスタリングする実験を行っています．K をいろいろ変えて実行し直して，結果を観察しよう．K-meansクラスタリングの結果は初期値に依存するので，実行するたびに結果が変わります．

---
### ［実験3］ クラスタリングを利用したカラー画像の減色

一般的なカラー画像は，一つの画素を赤緑青の3つの色で表すため，幅が $W$ 画素で高さが $H$ 画素の画像は $3WH$ 個の数値の集まりとなります．これらを $WH$ 個の 3 次元ベクトルの集まりとみなして $K$ 通りにクラスタリングすると，この画像を $K$ 通りの色だけで表す（減色する）ことができます．$K$-means クラスタリングでやってみましょう．

上の実験では NumPy を使って書いた $K$-means クラスタリングのプログラムを使っていましたが，ここでは Python の機械学習ライブラリ [scikit-learn](https://scikit-learn.org/) を使ってみることにします．

cf. [skpearn.cluster.KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

In [ ]:
import numpy as np
import cv2  # コンピュータビジョンライブラリ OpenCV
from sklearn.cluster import KMeans # scikit-learn の cluster パッケージの KMeans クラス
from google.colab.files import upload # Colab へのファイルのアップロード
from google.colab.patches import cv2_imshow # Colab 上に OpenCV 形式の画像を表示

次のセルを実行して，適当な画像ファイルをアップロードしましょう．ファイル名は，あらかじめ手元のPCで空白や日本語等を含まないものに変更しておくのがよいでしょう．

In [ ]:
# 画像をアップロード
rv = upload()
! ls

次のセルの1行目の `hoge.jpg` をアップロードしたファイルの名前に変更して実行しましょう．

In [ ]:
# 画像を読み込む．`hoge.jpg` を自分がアップロードしたファイルの名前に修正
fn = 'hoge.jpg'
img = cv2.imread(fn)

assert img is not None, f'画像 {fn} の読み込みに失敗？'
assert img.ndim == 3 and img.shape[2] == 3, '3チャンネルカラー画像のみ扱えます'
h, w = img.shape[:2]

maxSize = 640

# (長辺の長さ) <= maxSize にリサイズ
if max(w, h) > maxSize:
    if w >= h:
        w, h = maxSize, h*maxSize//w
    else:
        w, h = w*maxSize//h, maxSize
    img = cv2.resize(img, (w, h))

cv2_imshow(img)
print(f'img.shape = {img.shape}')

配列 `img` の形を変えた（ reshape した）ものを学習データとして $K$-means クラスタリングを実行します．

In [ ]:
# 学習データの配列 X を作る
X = img.reshape((img.shape[0]*img.shape[1], -1))
print(f'X.shape = {X.shape}')

K = 256 # クラスタ数

# KMeans クラスのインスタンスを生成
km = KMeans(n_clusters=K, n_init='auto', verbose=1)

# 学習を実行
km.fit(X)

出力表示の `Iteration` は学習の繰り返し回数，`inertia` は notebookB で説明している次の量（の定数倍）です．

$$
E = \sum_{k=1}^{K}\sum_{n:y_n = k} \Vert \mathbf{x}_n - \mathbf{c}_{k}\Vert^2
$$

学習の繰り返しによって減少していることが分かります．

それでは，この学習によって得られた結果を用いて実際に減色を実行してみましょう．

In [ ]:
# セントロイド（Kx3の2次元配列）の値を四捨五入して K 通りのカラーパレットを得る
centroid = (km.cluster_centers_ + 0.5).astype(int)
#print(centroid)

# 各画素値が割り振られたクラスタの番号を求める
lab = km.predict(X)

# 減色を実行
for ik in range(K):
    X[lab == ik, :] = centroid[ik]

# H x W x 3 の3次元配列に reshapeして画像として表示
img2 = X.reshape((img.shape[0], img.shape[1], 3))
cv2_imshow(img2)
print(f'K = {K}')

動作確認できたら，`K` の値をいろいろ変えたり，画像を変えたりしてみるとよいでしょう．

［よだんだよん］ ここでは，クラスタリングの学習に使った画像そのものを減色していますが，学習に用いる画像と減色する画像を別々にして実験してみるのも面白いです（コードの修正が必要）．